# Lab: Reconstructing Apple HDR Gain Maps

In this lab, we will manually reconstruct an HDR image from an SDR source and a Gain Map.
We will:
1. Load the SDR HEIC image.
2. Load the Gain Map PNG.
3. Apply the Apple Gain Map formula.
4. Save the result as a 16-bit PNG with PQ encoding and correct metadata.

In [ ]:
import numpy as np
import colour
import xml.etree.ElementTree as ET
from PIL import Image
import png
import io
import struct
import zlib
import os

# Ensure we can find the data folder
DATA_DIR = os.path.join(os.getcwd(), 'data')
print(f"Data directory set to: {DATA_DIR}")

**PNG Binary Helpers**<br>
Standard Python libraries often struggle to save 16-bit PNGs with specific HDR metadata (the cICP chunk). We will write a helper to manually inject this chunk, which tells the operating system (Windows/macOS) that the image uses the BT.2020 color primaries and the PQ transfer function

In [4]:
# Helper Functions (PNG & Metadata)
def create_png_chunk(chunk_type: bytes, data: bytes) -> bytes:
    """Creates a valid PNG chunk: [Length][Type][Data][CRC]"""
    chunk = struct.pack(">I", len(data)) + chunk_type + data
    crc = zlib.crc32(chunk_type + data) & 0xffffffff
    chunk += struct.pack(">I", crc)
    return chunk

def save_pq_png(filename, width, height, data_16bit):
    """Saves a 16-bit PNG with a cICP chunk for PQ HDR."""
    writer = png.Writer(width=width, height=height, bitdepth=16, greyscale=False)
    buffer = io.BytesIO()
    writer.write(buffer, data_16bit)
    png_bytes = buffer.getvalue()
    
    # cICP Chunk: Primaries=9 (BT.2020), Transfer=16 (PQ), Matrix=0, Range=1
    cicp_chunk = create_png_chunk(b'cICP', b'\x09\x10\x00\x01')
    
    with open(filename, 'wb') as f:
        f.write(png_bytes[:33])    # Signature and IHDR
        f.write(cicp_chunk)        # cICP Metadata
        f.write(png_bytes[33:])    # IDAT and IEND
    print(f"Saved: {filename}")

**Color Science & Metadata Logic**<br>
Here we handle the core logic. We need to:<br>

- *Parse XMP* : Apple stores the "Headroom" (how much brighter the HDR is compared to SDR) in XML metadata inside the image.<br>
- *Adapt Color Space* : Move from the phone's display space (P3) to the standard HDR space (BT.2020).


In [2]:
# Color Science Logic
def adaptation_to_bt2020(image_array, input_colorspace):
    return colour.RGB_to_RGB(
        image_array,
        input_colourspace=input_colorspace,
        output_colourspace=colour.models.RGB_COLOURSPACE_BT2020,
        chromatic_adaptation_transform="Bradford",
        apply_cctf_decoding=False,
        apply_cctf_encoding=False,
    )

def parse_headroom_from_xmp(pil_image):
    """Extracts the HDRGainMapHeadroom value from XMP metadata."""
    xmp_data = pil_image.info.get("xmp") or pil_image.info.get("XML:com.adobe.xmp")
    if not xmp_data:
        return 1.0 # Default

    try:
        namespaces = {
            "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
            "HDRGainMap": "http://ns.apple.com/HDRGainMap/1.0/"
        }
        root = ET.fromstring(xmp_data)
        target_tag = root.find(".//HDRGainMap:HDRGainMapHeadroom", namespaces)
        if target_tag is not None and target_tag.text:
            return float(target_tag.text)
    except Exception:
        pass
    return 1.0

**The Reconstruction Pipeline**<br>
This is the heart of the script. We implement the Apple Gain Map formula:<br>
$$L_{hdr} = L_{sdr} \times (1 + (Headroom - 1) \times GainMap)$$<br>

and finally, execute the conversion !

In [6]:
def apply_apple_gain_map(input_path, gain_map_path, ref_white, peak_nits):
    sdr_colorspace = colour.models.RGB_COLOURSPACE_DISPLAY_P3
    register_heif_opener()
    # 1. Load Images
    sdr_img = Image.open(input_path)
    sdr = np.array(sdr_img).astype(np.float32) / 255.0
    
    gain_map = Image.open(gain_map_path)
    gain_map_resized = gain_map.resize(sdr_img.size, resample=Image.BICUBIC)
    gain = np.array(gain_map_resized).astype(np.float32) / 255.0
    
    # Expand dimensions if needed
    if gain.ndim == 2: gain = np.stack([gain]*3, axis=-1)
    elif gain.shape[2] == 1: gain = np.concatenate([gain]*3, axis=-1)

    # 2. Linearize
    sdr_linear = sdr_colorspace.cctf_decoding(sdr)
    gain_linear = colour.models.oetf_inverse_BT709(gain)
    
    # 3. Apply Gain
    headroom = parse_headroom_from_xmp(gain_map)
    print(f"Headroom detected: {headroom}")
    hdr_linear_p3 = sdr_linear * (1.0 + (headroom - 1.0) * gain_linear)
    
    # 4. Convert to BT.2020 & PQ
    hdr_linear_2020 = adaptation_to_bt2020(hdr_linear_p3, sdr_colorspace)
    hdr_abs_nits = hdr_linear_2020 * ref_white
    hdr_pq = colour.models.eotf_inverse_BT2100_PQ(np.clip(hdr_abs_nits, 0, peak_nits))
    
    return hdr_pq

# Execution
# Define paths relative to the DATA_DIR constant
input_image_path = os.path.join(DATA_DIR, "112_115.HEIC")
gain_map_path = os.path.join(DATA_DIR, "112_115-urn_com_apple_photo_2020_aux_hdrgainmap.png")
output_filename = os.path.join(DATA_DIR, "output_112_115_PQ.png")

# Run the pipeline
if os.path.exists(input_image_path) and os.path.exists(gain_map_path):
    hdr_pq = apply_apple_gain_map(input_image_path, gain_map_path, 203, 1000)
    
    # Convert to 16-bit
    hdr_uint16 = (np.clip(hdr_pq, 0, 1) * 65535).astype(np.uint16)
    h, w, c = hdr_uint16.shape
    flat_rows = hdr_uint16.reshape(h, w * c)
    
    save_pq_png(output_filename, w, h, flat_rows)
else:
    print("Error: Files not found. Check your 'data' folder.")